In [10]:
#import Libraries
from tensorflow.keras.layers import Conv2D,Flatten,Dense,MaxPool2D,BatchNormalization,GlobalAveragePooling2D
from tensorflow.keras.applications.resnet50 import preprocess_input , decode_predictions
from tensorflow.keras.preprocessing.image import ImageDataGenerator ,load_img
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
import matplotlib as plt 
import numpy as np 
from tensorflow.keras import layers

In [2]:
#Define data and store it in variables
img_height, img_width = (224,224)
batch_size= 32
train_data = f"data/train"
val_data = f"data/val"
test_data = f"data/test"

In [4]:
#Divide into training, testing and validation
train_data_1 = ImageDataGenerator(preprocessing_function=preprocess_input,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True,
                                  validation_split=0.4)
train_genter = train_data_1.flow_from_directory(train_data,
                                               target_size=(img_height,img_width),
                                               batch_size=batch_size,
                                               class_mode="categorical",
                                               subset="training")
val_genter = train_data_1.flow_from_directory(val_data,
                                               target_size=(img_height,img_width),
                                               batch_size=batch_size,
                                               class_mode="categorical",
                                               subset="validation")

Found 1440 images belonging to 4 classes.
Found 320 images belonging to 4 classes.


In [5]:
test_genter = train_data_1.flow_from_directory(test_data,
                                               target_size=(img_height,img_width),
                                               batch_size=batch_size,
                                               class_mode="categorical",
                                               subset="validation")

Found 320 images belonging to 4 classes.


In [6]:
x,y = test_genter.next()
x.shape

(32, 224, 224, 3)

In [21]:
#Building a neural network and training it on images
model_cs = ResNet50(include_top=False,weights="imagenet")
x = model_cs.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024,activation="relu")(x)
pred = Dense(train_genter.num_classes,activation='softmax')(x)
model = Model(inputs=model_cs.input,outputs=pred)
for layer in model_cs.layers:
    layer.trainable = False
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(train_genter, epochs=2)

Epoch 1/2
45/45 [==============================] - 82s 2s/step - loss: 0.2532 - accuracy: 0.9375
Epoch 2/2
45/45 [==============================] - 80s 2s/step - loss: 0.0649 - accuracy: 0.9840


In [22]:
model.evaluate(test_genter)

10/10 [==============================] - 19s 2s/step - loss: 0.0786 - accuracy: 0.9875


[0.07858682423830032, 0.987500011920929]

In [14]:
cnn = Sequential([
                  layers.Conv2D(32, (2,2), activation='relu', input_shape=(224, 224, 3)),
                  layers.MaxPooling2D(3,3),
                  layers.Conv2D(64, kernel_size=(3,3),activation='relu',kernel_initializer='he_normal'),
                  layers.MaxPooling2D(2,2),
                  #layers.Conv2D(64, (2, 2), activation='relu'),
                  layers.Conv2D(64, kernel_size=(2,2),activation='relu',kernel_initializer='he_normal'),
                  layers.MaxPooling2D(2,2),
                  layers.Flatten(),
                  layers.Dense(128,activation='relu'),
                  layers.Dense(4,activation='softmax',kernel_initializer='glorot_normal')
                  
])

In [18]:
cnn.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
cnn.fit(train_genter, epochs=10)

Epoch 1/10
45/45 [==============================] - 30s 657ms/step - loss: 1.1345 - accuracy: 0.5319
Epoch 2/10
45/45 [==============================] - 30s 664ms/step - loss: 0.8920 - accuracy: 0.6250
Epoch 3/10
45/45 [==============================] - 30s 664ms/step - loss: 0.8105 - accuracy: 0.6750
Epoch 4/10
45/45 [==============================] - 30s 667ms/step - loss: 0.7651 - accuracy: 0.6986
Epoch 5/10
45/45 [==============================] - 30s 665ms/step - loss: 0.6994 - accuracy: 0.7167
Epoch 6/10
45/45 [==============================] - 30s 672ms/step - loss: 0.6100 - accuracy: 0.7569
Epoch 7/10
45/45 [==============================] - 31s 682ms/step - loss: 0.5786 - accuracy: 0.7861
Epoch 8/10
45/45 [==============================] - 31s 677ms/step - loss: 0.6253 - accuracy: 0.7569
Epoch 9/10
45/45 [==============================] - 30s 669ms/step - loss: 0.5094 - accuracy: 0.7951
Epoch 10/10
45/45 [==============================] - 31s 682ms/step - loss: 0.6532 - accura

In [20]:
cnn.evaluate(test_genter)

10/10 [==============================] - 4s 423ms/step - loss: 0.9041 - accuracy: 0.6594


[0.9041054844856262, 0.659375011920929]